## StarTracker Tracking drift check - 15-Mar-23

HyeYun Park - checking calibrated ra and dec during tracking

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u

In [ ]:
# function to read in the metadata that has been downloaded
def read_rubintv(fname):
    df = pd.read_json(fname).T
    df = df.dropna()
    times = df[['UTC','UTC wide']]
    
    df = df.drop(columns=['UTC','UTC wide'])
    df = df.astype(float)
    
    # set time vectors
    timenew = (today+'T'+ times['UTC'].to_numpy()).astype(np.datetime64)
    timenew2= (today+'T'+ times['UTC wide'].to_numpy()).astype(np.datetime64)
    
    df['UTC'] = pd.Series(timenew, index=df.index)
    df['UTC wide'] = pd.Series(timenew2, index=df.index)
    df['seq_num'] = df.index
    
    return df

In [ ]:
# read in dataframe
today = '2023-03-09'
fname = '~/data/startracker_%s.json'%today
df = read_rubintv(fname)

In [ ]:
df.columns

In [ ]:
df['Calculated Dec']*u.deg

In [ ]:
df['Calculated Dec wide']*u.deg

In [ ]:
df['MJD']

In [ ]:
time = Time(df['MJD'], format='mjd', scale='utc')

In [ ]:
data = df.iloc[:].reset_index() 

In [ ]:
data['Calculated Ra wide'].plot(color='lightgray')
plt.scatter(data.index, data['Calculated Ra wide'], cmap='tab20_r')

plt.ylabel('Ra', fontsize=16)
plt.xlabel('Seq Number', fontsize=16)
plt.savefig('pointing_ra.png',dpi=100)

In [ ]:
tracking_seq_number_start=2 #113 #249
tracking_seq_number_end=112 #230 #573
calculated_ra=data['Calculated Ra'][tracking_seq_number_start:tracking_seq_number_end]
calculated_dec=data['Calculated Dec'][tracking_seq_number_start:tracking_seq_number_end]
time_mjd=Time(data['MJD'][tracking_seq_number_start:tracking_seq_number_end], format='mjd', scale='utc')

In [ ]:
print (time_mjd.utc[0])

In [ ]:
print (time_mjd[0])

In [ ]:
time_mjd.format='fits'

In [ ]:
date=time_mjd.strftime('%Y %b %d')[0]
time=time_mjd.strftime('%H:%M:%S') #%d %b %Y')

In [ ]:
time_mjd.format='mjd'

In [ ]:
time_float=time_mjd.to_value('mjd', 'float')

In [ ]:
coef= np.polyfit(np.array(time_float), np.array(calculated_ra), 1)
poly1d_fn = np.poly1d(coef) 
coef_dec=np.polyfit(np.array(time_float), np.array(calculated_dec), 1)
poly1d_fn_dec=np.poly1d(coef_dec) 

In [ ]:
fig, (ax1, ax2,ax3) = plt.subplots(3,1, figsize=(10,20))
fig.subplots_adjust(wspace=0.)

ax1.scatter(np.array(time),np.array(calculated_ra),  s=25)
ax2.scatter(np.array(time),np.array(calculated_dec), s=25)

ax1.set_ylabel('Calculated Ra',fontsize=14)
ax1.set_xlabel('Time (utc)',fontsize=14)
ax1.xaxis.set_major_locator(plt.MaxNLocator(10))
ax1.plot(np.array(time), poly1d_fn(np.array(time_float)), '--k')
#ax1.set_ylim(np.min(calculated_ra)-0.001,np.max(calculated_ra)+0.001)
ax1.tick_params(axis='x', labelrotation=30)

ax2.set_ylabel('Calculated Dec',fontsize=14)
ax2.set_xlabel('Time (utc)',fontsize=14)
ax2.xaxis.set_major_locator(plt.MaxNLocator(10))
ax2.plot(np.array(time), poly1d_fn_dec(np.array(time_float)), '--k')
ax2.tick_params(axis='x', labelrotation=30)
#ax2.set_ylim(np.min(calculated_dec)-0.001,np.max(calculated_dec)+0.001)

ax3.plot(np.array(calculated_ra), np.array(calculated_dec))
ax3.scatter(np.array(calculated_ra), np.array(calculated_dec), color='r')
ax3.set_ylabel('Calculated Dec',fontsize=14)
ax3.set_xlabel('Calculated Ra',fontsize=14)

fig.suptitle('Calculated Ra&Dec %s'%date,fontsize=16)



In [ ]:
df.columns